# LangChain: Agents
Sometimes we think LLM as a learning model. Which memorize everthing and spirs out thing when we ask. 

But we can think LLM as a reasoning engine in whcih we ahve chunk of text and new information and do things what we ask. this can be done with agents.


## Outline:

* Using built in LangChain tools: DuckDuckGo search and Wikipedia
* Defining your own tools

In [3]:
# Setup environemnt varaibles and import stuffs that we will use 
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

import warnings
warnings.filterwarnings("ignore")

Note: LLM's do not always produce the same results. When executing the code in your notebook, you may get slightly different answers that those in the video.

In [4]:
# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

## Built-in LangChain tools

In [5]:
#!pip install -U wikipedia

In [6]:
from langchain.agents.agent_toolkits import create_python_agent
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
from langchain.tools.python.tool import PythonREPLTool
from langchain.python import PythonREPL
from langchain.chat_models import ChatOpenAI

Initalize langaugae model and set temperature =0. This is important because we are going to use LLM as a reasonig engine of an agent where it is connected to other data an sourses. So we want this reasonig engine to be as good precise. So we put temperature to zero to avaoid any randomness.

In [7]:
llm = ChatOpenAI(temperature=0, model=llm_model)

We use LLM math and wikipidea tool. \
llm-math toolsis a chain itself whcih uses a langauge model in conjunction with a calculator to do math problems.

wikipedia tool is an API whcih connects wikipidea allowing you to run search queries to run and search from wikipidea

In [8]:
tools = load_tools(["llm-math","wikipedia"], llm=llm)

initalise the agents with the tools, model type and agent tupe. \
here we use `CHAT_ZERO_SHOT_REACT_DESCRIPTION` agent type \
the imprtant thing to look at  is  'chat' agent is optimized to use with chat model and second thing is 'react' thsi is the prompting technique designed to get best reasonig from LLM models

In [9]:
agent= initialize_agent(
    tools, 
    llm, 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True, # when parsing not done we asked to pass back to LLM again and parse it.
    verbose = True)

In [10]:
# Ask agent to aks math question
agent("What is the 25% of 300?")



> Entering new AgentExecutor chain...
Thought: We can use the calculator tool to find 25% of 300.
Action:
```
{
  "action": "Calculator",
  "action_input": "25% of 300"
}
```
Observation: Answer: 75.0
Thought:Final Answer: 75.0

> Finished chain.


{'input': 'What is the 25% of 300?', 'output': '75.0'}

In [12]:
# Ask agent to aks math question
agent("What is sin 45 degree?")



> Entering new AgentExecutor chain...
Thought: Since the question is asking for the sine of 45 degrees, I can use a calculator to find the answer.

Action:
```
{
  "action": "Calculator",
  "action_input": "sin(45 degrees)"
}
```


Observation: Answer: 0.8509035245341183
Thought:Final Answer: 0.8509035245341183

> Finished chain.


{'input': 'What is sin 45 degree?', 'output': '0.8509035245341183'}

In [13]:
# Ask agent to aks math question
agent("What is the differentiation of sin(x) with respect to x")



> Entering new AgentExecutor chain...
Thought: We can use the Calculator tool to find the differentiation of sin(x) with respect to x.

Action:
```
{
  "action": "Calculator",
  "action_input": "differentiate sin(x)"
}
```



ValueError: LLMMathChain._evaluate("
sin(x)
") raised error: 'x'. Please try again with a valid numerical expression

## Wikipedia example

In [11]:
# Ask question from wikipisea
question = "Tom M. Mitchell is an American computer scientist \
and the Founders University Professor at Carnegie Mellon University (CMU)\
what book did he write?"
result = agent(question) 



> Entering new AgentExecutor chain...
Thought: I can use Wikipedia to find out which book Tom M. Mitchell wrote.
Action:
```
{
  "action": "Wikipedia",
  "action_input": "Tom M. Mitchell"
}
```
Observation: Page: Tom M. Mitchell
Summary: Tom Michael Mitchell (born August 9, 1951) is an American computer scientist and the Founders University Professor at Carnegie Mellon University (CMU). He is a founder and former chair of the Machine Learning Department at CMU. Mitchell is known for his contributions to the advancement of machine learning, artificial intelligence, and cognitive neuroscience and is the author of the textbook Machine Learning. He is a member of the United States National Academy of Engineering since 2010. He is also a Fellow of the American Academy of Arts and Sciences, the American Association for the Advancement of Science and a Fellow and past president of the Association for the Advancement of Artificial Intelligence. In October 2018, Mitchell was appointed as the 

In [14]:
# Ask question from wikipisea
question = "Do you know about Abhishek Saini who is a postdoc\
researh at Los Alamos National laboratory"
result = agent(question) 



> Entering new AgentExecutor chain...
Question: Do you know about Abhishek Saini who is a postdoc researcher at Los Alamos National Laboratory?
Thought: I should use Wikipedia to find information about Abhishek Saini.
Action:
```
{
  "action": "Wikipedia",
  "action_input": "Abhishek Saini postdoc researcher Los Alamos National Laboratory"
}
```

Observation: No good Wikipedia Search Result was found
Thought:I will try a different search query to find information about Abhishek Saini.
Action:
```
{
  "action": "Wikipedia",
  "action_input": "Abhishek Saini scientist"
}
```
Observation: Page: List of Punjabi people
Summary: Following is a list of famous and notable Punjabi people, an ethnic group belonging to the Punjab region. It contains people mainly from what is today Punjab, Pakistan and Punjab, India, and people with Punjabi ancestry or people who speak Punjabi as their primary language.

Page: Mission Mangal
Summary: Mission Mangal (transl. Mission Mars) is a 2019 Indian drama 

## Python Agent
You may have seen things like copilot, chatgpt with the code interpretter  plugin enabled.  They are using the language model to write code and executing that code. We will do that.

Lets create python agent and use same LLM

In [15]:
agent = create_python_agent(
    llm,
    tool=PythonREPLTool(),  # REPL is basically a way to interact with code, example jupyternotebook
    verbose=True
)

In [16]:
# Problem: sort the list of names
customer_list = [["Harrison", "Chase"], 
                 ["Lang", "Chain"],
                 ["Dolly", "Too"],
                 ["Elle", "Elem"], 
                 ["Geoff","Fusion"], 
                 ["Trance","Former"],
                 ["Jen","Ayai"]
                ]

In [17]:
# Ask agent
agent.run(f"""Sort these customers by \
last name and then first name \
and print the output: {customer_list}""") 

# We asked to print the output. These printed satement will send back to LLM so that it can 
# reason about output of the code that it ran



> Entering new AgentExecutor chain...
We can use the sorted() function in Python to sort the list of customers based on last name first and then first name.
Action: Use the sorted() function
Action Input: sorted([['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']], key=lambda x: (x[1], x[0]))
Observation: Use the sorted() function is not a valid tool, try another one.
Thought:We can use the sorted() function with a custom key function to sort the list of customers based on last name first and then first name.
Action: Use the sorted() function with a custom key function
Action Input: sorted([['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']], key=lambda x: (x[1], x[0]))
Observation: Use the sorted() function with a custom key function is not a valid tool, try another one.
Thought:We can use the sorted() function wi

"[['Jen', 'Ayai'], ['Harrison', 'Chase'], ['Lang', 'Chain'], ['Elle', 'Elem'], ['Trance', 'Former'], ['Geoff', 'Fusion'], ['Dolly', 'Too']]"

#### View detailed outputs of the chains

Lets look in deeper using langchain debug

In [19]:
import langchain
langchain.debug=True
agent.run(f"""Sort these customers by \
last name and then first name \
and print the output: {customer_list}""") 
langchain.debug=False

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "Sort these customers by last name and then first name and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]"
}
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
{
  "input": "Sort these customers by last name and then first name and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]",
  "agent_scratchpad": "",
  "stop": [
    "\nObservation:",
    "\n\tObservation:"
  ]
}
[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: You are an agent designed to write and execute python code to answer questions.\nYou have access to a python REPL, which you can use to execute python code.\nI

## Define your own tool
we can connect the agents with our own data tool api.

In [20]:
#!pip install DateTime

In [21]:
# 
from langchain.agents import tool  # can be apply to any tool
from datetime import date

In [25]:
# write a fucntion to print todays date
@tool # imprt tool decorator, This can be applied to any fucntions and turns it into tools that langchai can use.
def time(text: str) -> str:
    # In addition to name, write a detailed string, because this is what agent will use when we call this tool.
    """Returns todays date, use this for any \
    questions related to knowing todays date. \
    The input should always be an empty string, \
    and this function will always return todays \
    date - any date mathmatics should occur \
    outside this function."""
    return str(date.today())

In [28]:
# Create an agent and add time tool
agent= initialize_agent(
    tools + [time], 
    llm, 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

**Note**: 

The agent will sometimes come to the wrong conclusion (agents are a work in progress!). 

If it does, please try running it again.

In [27]:
try:
    result = agent("whats the date today?") 
except: 
    print("exception on external access")
    
# Note taht the action string is empty as we mentioned.



> Entering new AgentExecutor chain...
Thought: I can use the `time` tool to find out today's date.
Action:
```
{
  "action": "time",
  "action_input": ""
}
```

Observation: 2025-03-12
Thought:
Observation: Invalid or incomplete response
Thought:I will correct the previous action to get the correct date.
Action:
```
{
  "action": "time",
  "action_input": ""
}
``` 
Final Answer: 2025-03-12

> Finished chain.
